In [ ]:
# Import relevant libraries

!pip install spotipy

# Data augementation
!pip install numpy requests nlpaug

In [ ]:
import pandas as pd
import spotipy
from spotipy.oauth2 import SpotifyClientCredentials
from bs4 import BeautifulSoup
import requests

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import matplotlib.pyplot as plt

from torchtext.legacy import data as torchtext_data

# Data augmentation

import nlpaug.augmenter.char as nac
from nlpaug.util import Action

import nlpaug.augmenter.word as naw

In [ ]:
# Connect to Spotify

cid ='5e9160c5b94c4811ab7c239b3a36a460'
secret ='f8288b7c62564f25a5c7d94bd185e896'

client_credentials_manager = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager = client_credentials_manager)

In [ ]:
# Function to scrape lyrics from Genius

def get_album_tracks(uri_info, limit, offset=0):
  titles = []
  artists = []
  one = sp.playlist_tracks(uri_info, limit=limit, offset=offset, market='US')
  df1 = pd.DataFrame(one)
  for i, x in df1['items'].items():
    track = x['track']
    titles.append(track['name'])
    artists.append(track['artists'][0]['name'])
    df2 = pd.DataFrame({'title':titles,'artist':artists})
  return df2

def scrape_lyrics(artistname, songname):
  artistname2 = str(artistname.replace(' ','-')) if ' ' in artistname else str(artistname)
  songname2 = str(songname.replace(' ','-')) if ' ' in songname else str(songname)
  page = requests.get('https://genius.com/'+ artistname2 + '-' + songname2 + '-' + 'lyrics')
  html = BeautifulSoup(page.text, 'html.parser')

  lyrics1 = html.find("div", class_="lyrics")
  lyrics2 = html.find("div", class_="Lyrics__Container-sc-1ynbvzw-6 jYfhrf")
  if lyrics1:
    lyrics = lyrics1.get_text()
  elif lyrics2:
    lyrics = lyrics2.get_text()
  elif lyrics1 == lyrics2 == None:
    lyrics = None

  lines = []
  for div in html.findAll('div', {'class': 'Lyrics__Container-sc-1ynbvzw-6 jYfhrf'}):
    lines.extend([text if text[0] != '[' else ' ' for text in div.stripped_strings])
    
  lyrics = ""
  for line in lines:
    if line != ' ':
      lyrics += line + '@'  

  return lyrics


In [ ]:
# Scrape lyrics for the target genre

uri = 'spotify:playlist:5JDQq97ipoyekmMG5If3yc'
collections = [get_album_tracks(uri, 100, 0), 
               get_album_tracks(uri, 100, 100), 
               get_album_tracks(uri, 100, 200), 
               get_album_tracks(uri, 12, 300)]

lyrics = []
for df_tracks in collections:
  for index, row in df_tracks.iterrows():
      song = scrape_lyrics(row['artist'], row['title'])
      if song != '':
        lyrics.append([song])

In [ ]:
lyrics_copy = lyrics.copy()

substitute_aug = nac.RandomCharAug(action="substitute")
substitute_lyrics = []
for song in lyrics_copy:
  lines = song[0].split('@')
  aug_song = ""
  for line in lines:
    aug_song += (substitute_aug.augment(line) + "@")
  substitute_lyrics.append([aug_song])

swap_aug = nac.RandomCharAug(action="swap")
swap_lyrics = []
for song in lyrics_copy:
  lines = song[0].split('@')
  aug_song = ""
  for line in lines:
    aug_song += (swap_aug.augment(line) + "@")
  swap_lyrics.append([aug_song])

delete_aug = nac.RandomCharAug(action="delete")
delete_lyrics = []
for song in lyrics_copy:
  lines = song[0].split('@')
  aug_song = ""
  for line in lines:
    aug_song += (delete_aug.augment(line) + "@")
  delete_lyrics.append([aug_song])

# Word augmentation:

def word_aug(action):
  aug = None
  if action == "delete":
    aug = naw.RandomWordAug()
  else:
    aug = naw.RandomWordAug(action)
  aug_lyrics = []
  for song in lyrics_copy:
    lines = song[0].split('@')
    aug_song = ""
    for line in lines:  
      aug_song += (aug.augment(line) + "@")
    aug_lyrics.append([aug_song])
  return aug_lyrics

swap_word_lyrics = word_aug("swap")
delete_word_lyrics = word_aug("delete")

split_word_aug = naw.SplitAug()
split_word_lyrics = []
for song in lyrics_copy:
  lines = song[0].split('@')
  aug_song = ""
  for line in lines:
    aug_song += (split_word_aug.augment(line) + "@")
  split_word_lyrics.append([aug_song])


lyrics.extend(substitute_lyrics)
lyrics.extend(swap_lyrics)
lyrics.extend(delete_lyrics)

lyrics.extend(swap_word_lyrics)
lyrics.extend(delete_word_lyrics)
lyrics.extend(split_word_lyrics)

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import csv 

f_lyrics = open('/content/drive/MyDrive/csv_lyrics', 'w')
writer = csv.writer(f_lyrics)

for l in lyrics:
  writer.writerow(l)  

f_lyrics.close()

In [ ]:
f_overfit = open('/content/drive/MyDrive/csv_lyrics_overfit', 'w')
writer = csv.writer(f_overfit)

writer.writerow([lyrics[1]])  

f_overfit.close()

In [ ]:
# Count characters

chars = set()
for song in lyrics:
  for line in song:
    temp = list(set(line))
    for char in temp:
      chars.add(char)

chars.add("<BOS>")
chars.add("<EOS>")
chars = sorted(chars)
char_size = len(chars)

print(chars)
print("Number of unique characters: " + str(char_size))

In [ ]:
# Create dictionary of characters

char_to_index = dict((c, i) for i, c in enumerate(chars))
index_to_char = dict((i, c) for i, c in enumerate(chars))

print(char_to_index)
print(index_to_char)

In [ ]:
# Building the model

class SongGeneratorGRU(nn.Module):
  def __init__(self, char_size, embedding_size, hidden_size):
    super(SongGeneratorGRU, self).__init__()

    # Embedding layer
    self.embed = nn.Embedding(num_embeddings=char_size, embedding_dim=embedding_size)

    # RNN layer
    self.rnn = nn.GRU(input_size=embedding_size, hidden_size=hidden_size, batch_first=True)

    # Projection MLP layer
    self.mlp = nn.Linear(in_features=hidden_size, out_features=char_size)

  def forward(self, data, hidden=None):
    emb = self.embed(data)
    output, hidden = self.rnn(emb, hidden)
    output = self.mlp(output)
    return output, hidden


In [ ]:
 # Train with Teacher Forcing

def train(model, data, vocab_size, batch_size=1, num_epochs=1, lr=0.001, print_every=100):
    optimizer = torch.optim.Adam(model.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    it = 0
    
    losses, train_acc, valid_acc = [], [], []
    epochs = []

    data_iter = torchtext_data.BucketIterator(data, 
                                              batch_size=batch_size,
                                              sort_key=lambda x: len(x.text),
                                              sort_within_batch=True)
    
    for e in range(num_epochs):
        # get training set
        avg_loss = 0
        for (lyric, lengths), label in data_iter:
            target = lyric[:,1:]
            inp = lyric[:,:-1]
            # cleanup
            optimizer.zero_grad()
            # forward pass
            output, hidden = model(inp)
            loss = criterion(output.reshape(-1, vocab_size), target.reshape(-1))
            # backward pass
            loss.backward()
            optimizer.step()

            avg_loss += loss
            losses.append(float(loss)/lyric.size()[0])
            it += 1 # increment iteration count
            if it % print_every == 0:
                print("[Iter %d] Loss %f" % (it+1, float(avg_loss/print_every)))
                avg_loss = 0

    plt.title("Training Curve")
    plt.plot(losses, label="Train")
    plt.xlabel("Iterations")
    plt.ylabel("Loss")
    plt.show()

In [ ]:
text_field_overfit = torchtext_data.Field(sequential=True,      # text sequence
                                  tokenize=lambda x: x, # because are building a character-RNN
                                  include_lengths=True, # to track the length of sequences, for batching
                                  batch_first=True,
                                  use_vocab=True,       # to turn each character into an integer index
                                  init_token="<BOS>",   # BOS token
                                  eos_token="<EOS>")    # EOS token

fields_overfit = [('text', text_field_overfit)]
lyrics_overfit = torchtext_data.TabularDataset("/content/drive/MyDrive/csv_lyrics_overfit", "csv", fields_overfit)
text_field_overfit.build_vocab(lyrics_overfit)
vocab_stoi_overfit = text_field_overfit.vocab.stoi # so we don't have to rewrite sample_sequence
vocab_itos_overfit = text_field_overfit.vocab.itos # so we don't have to rewrite sample_sequence
vocab_size_overfit = len(text_field_overfit.vocab.itos)
print(vocab_size_overfit)
len(lyrics_overfit)

model = SongGeneratorGRU(vocab_size_overfit, 256, 256) #char_size, embedding_size, hidden_size
train(model, lyrics_overfit, vocab_size_overfit, batch_size=1, num_epochs=60, lr=0.004, print_every=10)

In [ ]:
text_field = torchtext_data.Field(sequential=True,      # text sequence
                                  tokenize=lambda x: x, # because are building a character-RNN
                                  include_lengths=True, # to track the length of sequences, for batching
                                  batch_first=True,
                                  use_vocab=True,       # to turn each character into an integer index
                                  init_token="<BOS>",   # BOS token
                                  eos_token="<EOS>")    # EOS token

fields = [('text', text_field)]
lyrics = torchtext_data.TabularDataset("/content/drive/MyDrive/csv_lyrics", "csv", fields)
text_field.build_vocab(lyrics)
vocab_stoi = text_field.vocab.stoi # so we don't have to rewrite sample_sequence
vocab_itos = text_field.vocab.itos # so we don't have to rewrite sample_sequence
vocab_size = len(text_field.vocab.itos)
print(vocab_size)
len(lyrics)

In [ ]:
model = SongGeneratorGRU(vocab_size, 256, 256) #char_size, embedding_size, hidden_size
train(model, lyrics, vocab_size, batch_size=120, num_epochs=95, lr=0.0045, print_every=50) 

In [ ]:
def sample_sequence(model, max_len=100, temperature=0.8):
    generated_sequence = ""
   
    inp = torch.Tensor([vocab_stoi["<BOS>"]]).long()
    hidden = None
    for p in range(max_len):
        output, hidden = model(inp.unsqueeze(0), hidden)
        # Sample from the network as a multinomial distribution
        output_dist = output.data.view(-1).div(temperature).exp()
        top_i = int(torch.multinomial(output_dist, 1)[0])
        # Add predicted character to string and use as next input
        predicted_char = vocab_itos[top_i]
        
        if predicted_char == "<EOS>":
            break
        generated_sequence += predicted_char       
        inp = torch.Tensor([top_i]).long()
    return generated_sequence.replace('@', '\n')

In [ ]:
# LET'S MAKE SOME MUSIC 🎵🎶

print(sample_sequence(model, max_len=1500, temperature=0.4))